In [ ]:
api_key = ""  # todo fix error with environ var

In [2]:
import csv

def calculate_openai_cost(tokens:list[tuple[int]], price_per_1M:tuple[float]):
    """calculate the cost
    param tokens: list of tuples, input and output
    param price_per_1M: tuple of pricing, input and output"""
    cost = 0
    for in_toks, out_toks in tokens:
        cost += in_toks*price_per_1M[0] + out_toks*price_per_1M[1]
    return cost/(10**6)  # don't forget to divide by 1M

def api_cost_logging(cost:float, experiment:str, date="", comment=""):
    new_row = [date, experiment, comment, cost]
    with open("api_costs.csv", mode='a', newline='', encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(new_row)


# Informal Tests

### Example texts

In [ ]:
# Small input, one FN is there, a second one is to be generated:
text1 = """<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file10224" type="Brief" source="HBBW-3" n="193">
	<teiHeader xml:lang="de">
		<fileDesc>
			<titleStmt>
				<title subtype="file">Konrad Geßner, Johannes Fries / Basel an Heinrich Bullinger, 25. Februar [1533]</title>
			</titleStmt>
			<publicationStmt>
				<authority>Universität Zürich</authority>
				</publicationStmt>
			<sourceDesc>
				</sourceDesc>
		</fileDesc>
		</teiHeader>
	<text xml:lang="la">
		<body>
			<div xml:id="div1" corresp="regest1">
				<p>
					<s n="1" xml:lang="la" type="auto">Optimo et integerrimo viro M. Henrico Bullingero, mecaenati charissimo.</s>
				</p>
				<p>
					<s n="2" xml:lang="la" type="auto">S.</s>
					<s n="3" xml:lang="la" type="auto">Impediunt nos ab itinere<note xml:id="fn3" type="footnote" n="3"><persName ref="p1283" cert="high">Geßner</persName> und <persName ref="p1214" cert="high">Fries</persName> befanden sich auf dem Weg nach <placeName ref="l59" cert="high">Bourges</placeName>.</note> nives, pluvia et ventorum vis.</s>
					<s n="4" xml:lang="la" type="auto"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName><note xml:id="fn4" type="footnote" n="4">"""

# Same Letter, but now the first FN is to be generated. Meaning the model has no direct example of a FN in the edition
text2 = """<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file10224" type="Brief" source="HBBW-3" n="193">
	<teiHeader xml:lang="de">
		<fileDesc>
			<titleStmt>
				<title subtype="file">Konrad Geßner, Johannes Fries / Basel an Heinrich Bullinger, 25. Februar [1533]</title>
			</titleStmt>
			<publicationStmt>
				<authority>Universität Zürich</authority>
				</publicationStmt>
			<sourceDesc>
				</sourceDesc>
		</fileDesc>
		</teiHeader>
	<text xml:lang="la">
		<body>
			<div xml:id="div1" corresp="regest1">
				<p>
					<s n="1" xml:lang="la" type="auto">Optimo et integerrimo viro M. Henrico Bullingero, mecaenati charissimo.</s>
				</p>
				<p>
					<s n="2" xml:lang="la" type="auto">S.</s>
					<s n="3" xml:lang="la" type="auto">Impediunt nos ab itinere<note xml:id="fn3" type="footnote" n="3">"""

# A long letter, the model has to generate the very last FN
text3 =  """<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file12260" type="Brief" source="HBBW-15" n="2213">
	<teiHeader xml:lang="de">
		<fileDesc>
			<titleStmt>
				<title subtype="file">Matthias Erb / Reichenweier an Heinrich Bullinger, 11. August [1545]</title>
			</titleStmt>
			<publicationStmt>
				<authority>Universität Zürich</authority>
				</publicationStmt>
			<sourceDesc>
				<bibl type="scan">Zürich StA</bibl>
				</sourceDesc>
		</fileDesc>
		</teiHeader>
	<text xml:lang="la">
		<body>
			<div xml:id="div1" corresp="regest1">
				<p>
					<s n="1" xml:lang="la" type="auto">Gratia domini tecum.</s>
					<s n="2" xml:lang="la" type="auto">Quum decima augusti<note xml:id="fn2" type="footnote" n="2">An einem Montag, dem Vortag (s. Z. 17f) des Datums dieses Briefes.</note> e suggestu descendissem me actutum a templo ad aulam conferens (vocatus scilicet a cancellario<note xml:id="fn3" type="footnote" n="3"><persName ref="p8103" cert="high">Sigismund Stier</persName>.</note> ob quędam negotia ecclesię componenda), venit hic tuus legatus<note xml:id="fn4" type="footnote" n="4"><persName ref="p18219" cert="high">Markus Wüst</persName>; s. oben <ref target="file12256">Nr. 2209</ref>, Anm. b.</note> hora ferme nona et, dum in angiportu sisto gradum, servum<note xml:id="fn5" type="footnote" n="5">Unbekannt.</note> ad limina de <persName ref="p1010" cert="high">Erbio</persName> rogat, me vero oculis ostiario significante ne proderet, statim digito demonstrat dicens: „Hic est.“</s>
					<s n="3" xml:lang="la" type="auto">Is ergo ad me veniens percunctatur, an mihi sit nomen <persName ref="p1010" cert="high">Erbio</persName>.</s>
					<s n="4" xml:lang="la" type="auto">Aio.</s>
					<s n="5" xml:lang="la" type="auto">Statimque litteras<note xml:id="fn6" type="footnote" n="6">Oben <ref target="file12256">Nr. 2209</ref>.</note> depromens dicebat a te missas.</s>
					<s n="6" xml:lang="la" type="auto">Acceptas legi simul obstupescens tuam, qui alias occupatissimus es, sedulitatem in describendis ex autographo copiis<note xml:id="fn7" type="footnote" n="7">Vgl. oben <ref target="file12256">Nr. 2209</ref>, 15-19.</note>.</s>
					<s n="7" xml:lang="la" type="auto">Stans ergo attonitus et propemodum exanimis iussi adolescentem expectare, dum redeam, atque subinde inter tractatus et varias dissidentium quęstiones et tacite et indesinenter volvo ac revolvo intime, quę scripseras.</s>
					<s n="8" xml:lang="la" type="auto">Absolutus a strepitu aulico eo cum adolescente pransum interque epulandum de singulis rebus percunctor, statim scilicet a mensa tibi responsurus.</s>
					<s n="9" xml:lang="la" type="auto">Dum id meditor, alius adest nuntius<note xml:id="fn8" type="footnote" n="8">Unbekannt.</note> litteras ferens a principe<note xml:id="fn9" type="footnote" n="9"><persName ref="p8098" cert="high">Graf Georg von Württemberg-Mömpelgard</persName>, der sich laut Z. 16 damals in <placeName ref="l569" cert="high">Worms</placeName> aufhielt. Seine Teilnahme am <placeName ref="l569" cert="high">Wormser</placeName> Reichstag ist in RTA JR XVIII-2 nicht belegt.</note>, in quibus multa mihi committuntur negotia principemque a comitiis<note xml:id="fn10" type="footnote" n="10">Der Reichstag in <placeName ref="l569" cert="high">Worms</placeName>.</note> propediem rediturum pollicentes.</s>
					<s n="10" xml:lang="la" type="auto">Hinc eo die, tametsi adolescens crebro urgeret abituriens, nil scripsi omnia in posterum, hoc est in hodiernum, reiiciens diem.</s>
					<s n="11" xml:lang="la" type="auto">Haec de pueri tui diligentia.</s>
				</p>
				<p>
					<s n="12" xml:lang="la" type="auto">Quod vero ad negotium adtinet, paucis haec accipe.</s>
					<s n="13" xml:lang="la" type="auto">Fateor me iam annum quinquagesimum egressum<note xml:id="fn11" type="footnote" n="11"><persName ref="p1010" cert="high">Erb</persName> war 1494 geboren.</note>, et qui nunc in ea aetate sum, quę sedatior esse cupit, sepicule curas aulicas, quibus distringor,<pb type="scan" next="257v."/> depręcatum, sed parum promovisse; imo, quo plus me expedire cupio, eo amplius perstringor dispendio et vitae et rerum.</s>
					<s n="14" xml:lang="la" type="auto">Sic deo optimo maximo visum est.</s>
				</p>
				<p>
					<s n="15" xml:lang="la" type="auto">Quod ad vitam, ut omnia manifestius depingam, subit tarda senectus,<note xml:id="fn12" type="footnote" n="12"><hi>Vergil,</hi> Aeneis, 9, 610.</note> quę difficilis incipit esse, sola non suffitiens laboribus, dum indies creberrime exantlatur.</s>
					<s n="16" xml:lang="la" type="auto">Nullum habeo symmistam, ut nosti, opinor.</s>
					<s n="17" xml:lang="la" type="auto">Plures iussu principis vocavi; sed horrent id, quod ego quęror.</s>
					<s n="18" xml:lang="la" type="auto">Metuunt enim sibi ab ingeniis cervinis<note xml:id="fn13" type="footnote" n="13">ingenium cervinum: ein ängstliches Wesen; s. <bibl>Adagia</bibl>, 2, 7, 36 (<bibl>ASD</bibl> II/4 110, <ref target="file11679">Nr. 1636</ref>).</note>, nec ego quenquam nisi volentem ad me trahere volo, etc.<pb type="edition" next="439"/></s>
					<s n="19" xml:lang="la" type="auto">Quod vero ad res meas adtinet, male collocat ille, quem ex nomine nosti, stipendia.</s>
					<s n="20" xml:lang="la" type="auto">Ex patrimonio intra quinquennium<note xml:id="fn16" type="footnote" n="16">D.h. seit 1539, was mit dem Amtsantritt <persName ref="p17986" cert="high">Henmans</persName> übereinstimmt.</note> ducentos fere aureos non in meam, sed magis ecclesię necessitatem exposui, quod dies domini revelabit.</s>
					<s n="21" xml:lang="la" type="auto">Nec quęror, optime frater, de perditis rebus, sed de Harpiis furacibus, qui recte monentes non exaudiunt et ecclesię avaritię notam inurunt; quod non adeo magni referret, nisi metuendum de excidio foret.</s>
					<s n="22" xml:lang="la" type="auto">Quamdiu rapiendum, placet Christus; dum defecerit ecclesię fiscus, non placebit ultra Christus!</s>
					<s n="23" xml:lang="la" type="auto">Hactenus 3 fuimus in isto oppido, ubi quisque designatum habuit<pb type="edition" next="440"/> stipendium.</s>
					<s n="24" xml:lang="la" type="auto">Ego nunc solus omnibus curis pręfectus aegre a satrapis<note xml:id="fn18" type="footnote" n="18">Zu verstehen: von den Beamten, u.a. vom neuen Schaffner.</note> meum recipio stipendium, et si egerem, vix triobulo sublevarent.</s>
					<s n="25" xml:lang="la" type="auto">Sunt pręterea et alię rationes, quibus te nolo nunc gravare scribendo, etc.</s>
					<s n="26" xml:lang="la" type="auto">Atque inter tot turbines nondum video satis iustam occasionem desertionis huius ecclesię.</s>
				</p>
				<p>
					<s n="27" xml:lang="de" type="auto"><pb type="scan" next="257a,r."/>Summa.</s>
					<s n="28" xml:lang="la" type="auto">Ita haereo inter sacrum et saxum<note xml:id="fn19" type="footnote" n="19">Siehe Adagia, 1, 1, 15 (<bibl>ASD</bibl> II/1 128f, Nr. 15).</note>, ut nunc sine discrimine ecclesięque Christi maxima iactura, ut facile intelligis, discedere nequeam.</s>
				</p>
				<p>
					<s n="29" xml:lang="la" type="auto">Atque eę sunt potissimę rationes.</s>
					<s n="30" xml:lang="la" type="auto">Ecclesias nostras iam a sexto anno<note xml:id="fn20" type="footnote" n="20">D.h. seit 1538, als Erbin Reichenweier tätig wurde.</note> utcunque in ordinem, ut potuimus - quando, ut volueramus, non licuit - redegimus.</s>
				</p>
				<p>
					<s n="31" xml:lang="la" type="auto">Annitendum ergo, ut cępta perficiantur.</s>
					<s n="32" xml:lang="la" type="auto">Ad hoc ecclesia illa nobis contigua, Montbelgardiana, cum suo principe<note xml:id="fn21" type="footnote" n="21"><persName ref="p1546" cert="high">Graf Christoph von Württemberg</persName>.</note> adhuc vacillat; de qua totus liber foret perscribendus.</s>
					<s n="33" xml:lang="la" type="auto">Pręterea scholę quattuor in isto parviusculo agro sunt constitutę et quinta in rure instituenda, stipendia quoque studiosis quibusdam ante paucos menses pollicita,<note xml:id="fn23" type="footnote" n="23">Erst 1555 kam es zur Einrichtung eines solchen Stipendiums durch Graf Georg; s. <hi>Adam,</hi> aao, S. 306.</note> sed haec non omnia firmata et rata, proponenda etiamnum ecclesię quędam decreta, quibus ad disciplinam et sanctos mores utcunque redigantur, idque statim redeunte principe.</s>
					<s n="34" xml:lang="la" type="auto">Illo enim absente haec omnia conscripta sunt.</s>
					<s n="35" xml:lang="la" type="auto">Dum igitur nunc mutavero locum omnia sub cultro relinquens<note xml:id="fn24" type="footnote" n="24">omnia sub cultro relinquens: alles der Gefahr aussetzend; vgl. <bibl>Adagia</bibl>, 2, 10, 83 (<bibl>ASD</bibl> II/4 328, <ref target="file12030">Nr. 1983</ref>).</note>, facile intelligis, quid periculi inmineat.</s>
					<s n="36" xml:lang="la" type="auto">In primis severus princeps, a quo omnia pendent, facile vel mutaretur vel irritaretur, ut solet hoc hominum genus; id quod nimirum magis in ecclesię quam in meum incomodum cęderet.</s>
					<s n="37" xml:lang="la" type="auto">Viden inter coelum et terram me suspensum palpitare neque posse hac ratione vel vocatione, quę tamen et sancta et iusta, liberari?</s>
					<s n="38" xml:lang="la" type="auto">Occasio mihi expectanda comodior, quę me liberet sine ecclesię iactura.</s>
					<s n="39" xml:lang="la" type="auto">Nam si iam abirem, ecclesia viduata absque pastore inter cunctatores et cessatores fluctuaret, etc.</s>
				</p>
				<p>
					<s n="40" xml:lang="la" type="auto"><pb type="scan" next="257a,v."/>Ago vero et habeo tibi gratias inmortales, humanissime Bullingere, quod admodum ingenti cura meo nomine laboraveris, atque hoc impensius te amo, quod charitatis offitium non fucate, sed dextre candideque in me collocaris, imo significaris in hac sancta vocatione.</s>
					<s n="41" xml:lang="la" type="auto">Respondebit olim procul dubio deus optimus maximus tuis votis, quo <placeName ref="l1976" cert="high">Augustani</placeName> nanciscantur virum aliquem sanctum, ecclesię dignum ministrum,<note xml:id="fn25" type="footnote" n="25"><persName ref="p8018" cert="high">Johannes Haller</persName> wurde schließlich aus Zurich nach <placeName ref="l1976" cert="high">Augsburg</placeName> gesandt; s. oben <ref target="file12211">Nr. 2164</ref>, Anm. 10.</note> atque ego etiam tuis assiduis<pb type="edition" next="441"/> praecibus vel liberer publica occasione vel sub onere non defitiam.</s>
					<s n="42" xml:lang="la" type="auto">Sed te etiamnum per Iesum Christum, ne deinceps plus mihi vel tribuas, immodicus scilicet virtutum mearum, quibus penitus careo, pręco, vel magis sentias de me, quam res est.</s>
					<s n="43" xml:lang="la" type="auto">Optime enim mihi ipsi conscius sum curtę suppellectilis,<note xml:id="fn26" type="footnote" n="26"><hi>Persius,</hi> Saturae, 4, 52.</note> quam tu, ni fallor, maiori atque ego, nescio quorum iuditio persuasus, aestimas.</s>
				</p>
				<p>
					<s n="44" xml:lang="la" type="auto">Porro, quod nunc temporis importunitas negat, alias benignus deus, ecclesiarum verus et unicus pastor<note xml:id="fn27" type="footnote" n="27">Vgl. <cit type="bible"><ref>Hebr 13, 20</ref></cit>; <cit type="bible"><ref>1Petr 2, 25</ref></cit>; <cit type="bible"><ref>5, 4.</ref></cit></note>, solita clementia sancta resarciet oportunitate.</s>
					<s n="45" xml:lang="la" type="auto">Sancto quoque magistratui <placeName ref="l1976" cert="high">Augustano</placeName>, inprimis vero erudito viro domino <persName ref="p8354" cert="high">Georgio Leto</persName><note xml:id="fn28" type="footnote" n="28"><persName ref="p8354" cert="high">Georg Frölich</persName>.</note>, ingentes gratias meo nomine agas, obsecro, quod me, ineptum homuntionem,<note xml:id="fn29" type="footnote" n="29">Vgl. <cit type="bible"><ref>1Kor 15,9</ref></cit>; <cit type="bible"><ref>Eph 3, 8</ref></cit>.</note> adeo egregia dignati sint vocatione.</s>
					<s n="46" xml:lang="la" type="auto">Ego quoque deinceps eius ecclesię maiorem habebo - si potis fuero - ubique rationem, optoque illis stabilem pacem et concordiam talesque antistites, qui pietate, eruditione, prudentia, vigilantia et fidelitate ita pręmineant, quo Christus in ea republica ita mittat radices, ut nullis satanę et impiorum imposturis deiiciatur.</s>
					<s n="47" xml:lang="la" type="auto">Monebis ergo illos, ne me deinceps expectent; ignoro enim meam liberationem.</s>
					<s n="48" xml:lang="la" type="auto">Servet te Christus.</s>
				</p>
				<p>
					<note type="attachment">[Beilage:]</note>
					<s n="49" xml:lang="la" type="auto"><pb type="scan" next="258r."/>Nova: Absolutionem conmitiorum<note xml:id="fn30" type="footnote" n="30">Der Reichstag zu <placeName ref="l569" cert="high">Worms</placeName>.</note> non ignoras, in quibus nil certi definitum, nisi quod istis hybernis mensibus adversarii nobis pacem condixerunt; futura aestate nobis providendum erit<note xml:id="fn31" type="footnote" n="31">Ein Gerücht.</note>.</s>
					<s n="50" xml:lang="la" type="auto">Acta comitiorum nondum sunt publicata.</s>
					<s n="51" xml:lang="la" type="auto">Aiunt quidam caesari<note xml:id="fn33" type="footnote" n="33"><persName ref="p18538" cert="high">Karl V.</persName></note> omnia esse commissa, ut dicat sententiam etiam in religionis negotio certis tamen conditionibus; quodsi verum fuerit, ovem dicemus lupo<note xml:id="fn34" type="footnote" n="34">Vgl. <cit type="bible"><ref>Joh 10, 12</ref></cit>.</note> commissam.</s>
				</p>
				<p>
					<s n="52" xml:lang="la" type="auto"><persName ref="p18477" cert="high">Ferdinandus</persName> priori septimana<note xml:id="fn35" type="footnote" n="35">Zu verstehen: In der ersten Augustwoche. - <persName ref="p18477" cert="high">Ferdinand I.</persName> war am 31. Juli 1545 aus <placeName ref="l569" cert="high">Worms</placeName> abgereist; s. unten <ref target="file12261">Nr. 2214</ref>, 28 und Anm. 20.</note> a comitiis magno strepitu solvens Bohemiam petit cum <persName ref="p7688" cert="high">Nausea</persName><note xml:id="fn36" type="footnote" n="36"><persName ref="p7688" cert="high">Friedrich Nausea</persName>, Bischof von <placeName ref="l546" cert="high">Wien</placeName>.</note> suo et quibusdam monachis.</s>
					<s n="53" xml:lang="la" type="auto">Sequenti septimana<pb type="edition" next="442"/> caesar abiturus est, sed quo, ignoratur<note xml:id="fn37" type="footnote" n="37"><persName ref="p18538" cert="high">Karl V.</persName> hatte <placeName ref="l569" cert="high">Worms</placeName> am 7. August in Richtung <placeName ref="l267" cert="high">Köln</placeName> verlassen; s. oben <ref target="file12261">Nr. 2214</ref>, 31 und Anm. 23f.</note>.</s>
					<s n="54" xml:lang="la" type="auto">Erant alia dicta comitia ad <placeName ref="l410" cert="high">Reginospurgum</placeName> ad mensem octobrem,<note xml:id="fn38" type="footnote" n="38">Der geplante Reichstag in <placeName ref="l410" cert="high">Regensburg</placeName>, der erst Anfang 1546 begann; s. oben <ref target="file12258">Nr. 2211</ref>, Anm. 24.</note>, sed aiunt quidam irritata<note xml:id="fn39" type="footnote" n="39">Hier im Sinne von "vergeblich"; s. <hi><persName ref="p8320" cert="high">Dasypodius</persName>,</hi> Dic. 199.</note>, etc.</s>
				</p>
				<p>
					<s n="55" xml:lang="la" type="auto">Urgent plures nacionale concilium, idque <placeName ref="322" type="auto_name">Metis</placeName><note xml:id="fn40" type="footnote" n="40"><placeName ref="l322" cert="high">Metz</placeName>.</note> celebrari postulant.</s>
					<s n="56" xml:lang="la" type="auto">Musher<note xml:id="fn41" type="footnote" n="41">Vom französischen "Monsieur".</note> de Gyß<note xml:id="fn42" type="footnote" n="42"><persName ref="p18450" cert="high">Claude I. von Lothringen</persName>, Herzog von Guise.</note>, defuncti ducis Lotharingię frater<note xml:id="fn43" type="footnote" n="43">Der am 12. Juni 1545 verstorbene <persName ref="p7637" cert="high">François I. von Lothringen</persName> war nicht Claudes Bruder, sondern sein Neffe. Bruder von Claude war der Vater des Verstorbenen, nämlich <persName ref="p18431" cert="high">Anton II. von Lothringen</persName> (gest. Juni 1544).</note>, patruum suum conscripto milite et auxilio regis<note xml:id="fn45" type="footnote" n="45"><persName ref="p7637" cert="high">König Franz I.</persName></note> Gallorum conatur e ducatu pellere.</s>
					<s n="57" xml:lang="la" type="auto"><persName ref="p18538" cert="high">Caesar</persName> vero iuniori<note xml:id="fn46" type="footnote" n="46"><persName ref="p18536" cert="high">Karl III. von Lothringen</persName>.</note>, qui patri<note xml:id="fn47" type="footnote" n="47"><persName ref="p7637" cert="high">François I. von Lothringen</persName>.</note> successit, subsidium et <placeName ref="l29" cert="high">Hispanum</placeName> militem pollicitus est.</s>
					<s n="58" xml:lang="la" type="auto">Quodsi res non fuerit composita, putant novum bellum inter caesarem et Gallum oriturum.</s>
					<s n="59" xml:lang="la" type="auto">Feruntur alia quedam nova, sed forte non fide digna: mundus mendatiis scatet<note xml:id="fn48" type="footnote" n="48">Vgl. <bibl>TPMA</bibl> XIII 46, 125f: Die Welt ist falscher Zungen voll.</note>.</s>
					<s n="60" xml:lang="la" type="auto">Haec omnia raptim.</s>
				</p>
				<p>
					<s n="61" xml:lang="la" type="auto">Valebis igitur, vir humanissime, cum ecclesia sancta et familia.</s>
					<s n="62" xml:lang="la" type="auto">Salutabis vicissim tuos, in primis meum <persName ref="p8412" cert="high">Megandrum</persName>, <persName ref="p8437" cert="high">Erasmum</persName>,<note xml:id="fn49" type="footnote" n="49">"""

## generate

#### test 1
Formulating the completion task in Chat template.

Original FN: <br>
```xml
<persName ref="p1214" cert="high">Fries</persName>
und
<persName ref="p1283" cert="high">Geßner</persName>
waren in einem Brief
<persName ref="p8127" cert="high">Pellikans</persName>
vom 19. Februar 1533 an
<persName ref="p8418" cert="high">Myconius</persName>
empfohlen worden (Zürich StA, E II 358, 98).
</note>
```
FN from the ICL:<br>
```xml
<note xml:id="fn49" type="footnote" n="49">
<persName ref="p8437" cert="high">Erasmus Schmid</persName>
.
</note>
```

In [21]:


ICL_SYSTEM_PROMPT = "Du bist Historiker und hast dich auf die Reformation spezialisiert. Gerade arbeitest du daran Briefe von Heinrich Bullinger zu edieren. Vervolständige die Inhaltlichen Fussnoten."

# footnote for the icl
footnote_text2 = """<note xml:id="fn3" type="footnote" n="3"><persName ref="p1283" cert="high">Geßner</persName> und <persName ref="p1214" cert="high">Fries</persName> befanden sich auf dem Weg nach <placeName ref="l59" cert="high">Bourges</placeName>.</note>"""

# Note the openai API does not allow for arbitrary names for the chatting parties:
# {'error': {'message': "Invalid value: 'historian'. Supported values are: 'system', 'assistant', 'user', 'function', and 'tool'.", 'type': 'invalid_request_error', 'param': 'messages[1].role', 'code': 'invalid_value'}}

def icl_prompt(text):
  """Generate a prompt that gives the example from text2"""
  messages = [
      {"role": "system", "content": ICL_SYSTEM_PROMPT},
      {"role": "user", "content": text2},   
      {"role": "assistant", "content": footnote_text2},
      {"role": "user", "content": text}
  ]
  return messages




In [24]:
model = OpenAiModels("gpt-4o-mini-2024-07-18", api_key)
tokens = []
for i in range(10):
    text, in_toks, out_toks = model.generate(icl_prompt(text1))
    tokens.append((in_toks, out_toks))
    print(text)


<note xml:id="fn4" type="footnote" n="4">Myconius, ein wichtiger Reformator und Freund von Bullinger, war in Basel tätig und spielte eine zentrale Rolle in der Verbreitung der reformatorischen Ideen in der Region.</note>
<note xml:id="fn4" type="footnote" n="4">Myconius, ein wichtiger Reformator und Freund von Bullinger, war in Basel tätig und spielte eine zentrale Rolle in der Verbreitung der reformatorischen Ideen in der Region.</note>
<note xml:id="fn4" type="footnote" n="4">Myconius, ein wichtiger Reformator und Freund von Bullinger, war in Basel tätig und spielte eine bedeutende Rolle in der Verbreitung der reformatorischen Ideen in der Region.</note>
<note xml:id="fn4" type="footnote" n="4">Myconius, ein wichtiger Reformator und Freund von Bullinger, war in Basel tätig und spielte eine zentrale Rolle in der Verbreitung der reformatorischen Ideen in der Region.</note>
<note xml:id="fn4" type="footnote" n="4">Myconius, ein wichtiger Reformator und Freund von Bullinger, war in Basel

In [25]:
calculate_openai_cost(tokens, (0.3,1.2))

0.0033066000000000003

#### Test 2
leaving one footnote out. Not optimal result and also more expensive?

In [26]:
import re
import pandas as pd

def footnote_regex(n):
  matching_string = (fr"( ?<note [^>]*? type=\"footnote\" n=\"{n}\">)" # matching group 1: the opening tag
                  r"(.*?(?=<\/note>))"  # matching group 2 everything up until the closing tag (positive lookup!! no matching group!!)
                  r"(<\/note>)"  # matching group 3: the endtag
  )
  return matching_string

def remove_footnote_content(text, n):
  """remove all content from the footnote n="n" """
  return re.sub(footnote_regex(n), r"\1\3", text)

def get_footnote_content(text, n):
  """get the content of a FN"""
  return re.search(footnote_regex(n), text).group(2)


ICL_SYSTEM_PROMPT = "Du bist Historiker und hast dich auf die Reformation spezialisiert. Gerade arbeitest du daran Briefe von Heinrich Bullinger zu edieren. Vervolständige die Inhaltlichen Fussnoten."

def HISTORIAN_PROMPT(text, n):
  return f"Bitte schlage mir einen Text für Fussnote n={n} in folgendem Dokument vor:\n\n{text}"

def instruct_prompt_fill_in(letter, n, example=(11130, 7)):
  """
  param text: Letter TEI
  param n: Footnote to generate
  example: letter_id and FN to give as an example, 12342 is from the dev-set..."""

  # find a good example letter, to put as icl prompt...
  # get the letter
  example_letter_id, example_n = example
  with open(f"../../data/downsized_letters/{example_letter_id}.xml") as infile:
    example_letter = infile.read()

  messages = [
      {"role": "system", "content": ICL_SYSTEM_PROMPT},
      {"role": "user", "content": HISTORIAN_PROMPT(remove_footnote_content(example_letter, example_n), example_n)},
      {"role": "assistant", "content": get_footnote_content(example_letter, example_n)},
      {"role": "user", "content": HISTORIAN_PROMPT(letter, n)}
  ]

  return messages



# test letter
letter_id = 10224


footnote_df = pd.read_csv("../../data/footnote_downsized_df.csv")
# get the letter
with open(f"../../data/downsized_letters/{letter_id}.xml") as infile:
  letter = infile.read()


In [14]:
instruct_prompt_fill_in(letter, 3)

[{'role': 'system',
  'content': 'Du bist Historiker und hast dich auf die Reformation spezialisiert. Gerade arbeitest du daran Briefe von Heinrich Bullinger zu edieren.'},
 {'role': 'user',
  'content': 'Bitte schlage mir einen Text für Fussnote n=7 in folgendem Dokument vor:\n\n<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file11130" type="Brief" source="HBBW-8" n="1087">\n\t<teiHeader xml:lang="de">\n\t\t<fileDesc>\n\t\t\t<titleStmt>\n\t\t\t\t<title subtype="file">Andreas Bodenstein an Heinrich Bullinger, [Anfang] Januar 1538</title>\n\t\t\t</titleStmt>\n\t\t\t<publicationStmt>\n\t\t\t\t<authority>Universität Zürich</authority>\n\t\t\t\t</publicationStmt>\n\t\t\t<sourceDesc>\n\t\t\t\t<bibl type="scan">Zürich StA</bibl>\n\t\t\t\t</sourceDesc>\n\t\t</fileDesc>\n\t\t</teiHeader>\n\t<text xml:lang="la">\n\t\t<body>\n\t\t\t<div xml:id="div1" corresp="regest1">\n\t\t\t\t<p>\n\t\t\t\t\t<s n="1" xml:lang="la" type="auto">S.</s>\n\t\t\t\t\t<s n="2" xml:lang="la" type="auto">Tot obsequiis 

In [15]:
tokens = []
for i in range(10):
    text, in_toks, out_toks = model.generate(instruct_prompt_fill_in(letter, 3))
    tokens.append((in_toks, out_toks))
    print(text)

Die Reise von Geßner und Fries nach Bourges war Teil ihrer Bemühungen, an der dortigen Universität zu studieren und sich mit anderen Gelehrten auszutauschen. Bourges war zu dieser Zeit ein Zentrum humanistischer Studien und zog viele Intellektuelle an, die sich mit den reformatorischen Ideen auseinandersetzten.
Die Reise von Geßner und Fries nach Bourges war Teil ihrer Bemühungen, an der dortigen Universität zu studieren und sich mit anderen Gelehrten auszutauschen. Bourges war zu dieser Zeit ein Zentrum humanistischer Studien und zog viele Intellektuelle an, die sich mit den Ideen der Reformation und der klassischen Antike auseinandersetzten.


In [16]:
calculate_openai_cost(tokens, (0.3,1.2))

0.0024186

#### Test 3
giving the empty letter, with one FN as an example

In [27]:
def instruct_prompt_no_fns(letter, n, example_n=None):
  """
  param text: Letter TEI
  param n: Footnote to generate
  example: letter_id and FN to give as an example, 12342 is from the dev-set..."""

    # footnotes of that letter
  footnote_df_letter = footnote_df[footnote_df["letter_id"] == letter_id]

  letter_no_fns = letter

  # removing all FNs
  for footnote in footnote_df_letter.iloc:  # Maybe we'll have to take care of the labels here...
    letter_no_fns = remove_footnote_content(letter_no_fns, footnote["n_footnote"])


  messages = [
      {"role": "system", "content": ICL_SYSTEM_PROMPT},
  ]

  # add example if specified
  if example_n:
    messages.append({"role": "user", "content": HISTORIAN_PROMPT(letter_no_fns, example_n)},)
    messages.append({"role": "assistant", "content": get_footnote_content(letter, example_n)})

  # add prompt for generation
  messages.append({"role": "user", "content": HISTORIAN_PROMPT(letter_no_fns, n)})

  return messages

zero shot, wacky

In [50]:
tokens = []
for i in range(10):
    text, in_toks, out_toks = model.generate(instruct_prompt_no_fns(letter, 3))
    tokens.append((in_toks, out_toks))
    print(text)

Für die Fußnote n=3 könnte folgender Text vorgeschlagen werden:

"Die widrigen Wetterbedingungen, insbesondere Schnee, Regen und starker Wind, hindern uns an der Weiterreise."
Für die Fußnote n=3 könnte folgender Text vorgeschlagen werden:

"Die widrigen Wetterbedingungen, insbesondere Schnee, Regen und starker Wind, hindern uns an der Weiterreise."
Für die Fußnote n=3 könnte folgender Text vorgeschlagen werden:

"Die widrigen Wetterbedingungen, insbesondere Schnee, Regen und starker Wind, hindern uns an der Weiterreise."
Für die Fußnote n=3 könnte folgender Text vorgeschlagen werden:

"Die widrigen Wetterbedingungen, insbesondere Schnee, Regen und starker Wind, hindern uns an der Weiterreise."
Für die Fußnote n=3 könnte folgender Text vorgeschlagen werden:

"Die widrigen Wetterbedingungen, insbesondere Schnee, Regen und starker Wind, hindern uns an der Weiterreise."
Für die Fußnote n=3 könnte folgender Text vorgeschlagen werden:

"Die widrigen Wetterbedingungen, insbesondere Schnee, R

In [51]:
calculate_openai_cost(tokens, (3,6))

0.03447

In [28]:
tokens = []
for i in range(10):
    text, in_toks, out_toks = model.generate(instruct_prompt_no_fns(letter, 4, example_n=3))
    tokens.append((in_toks, out_toks))
    print(text)
    

<note xml:id="fn4" type="footnote" n="4">Myconius, ein bekannter Humanist und Reformator, war zu dieser Zeit in Basel aktiv und spielte eine wichtige Rolle in der reformatorischen Bewegung. Die Erwähnung seiner Person deutet auf die enge Verbindung zwischen den Reformatoren und den humanistischen Kreisen in Basel hin.</note>
<note xml:id="fn4" type="footnote" n="4">Myconius, ein bekannter Humanist und Reformator, war zu dieser Zeit in Basel aktiv und spielte eine wichtige Rolle in der reformatorischen Bewegung. Die Erwähnung seiner Wohnstätte deutet auf die enge Verbindung zwischen den Reformatoren und den humanistischen Kreisen in Basel hin.</note>
<note xml:id="fn4" type="footnote" n="4">Myconius, ein bekannter Humanist und Reformator, war zu dieser Zeit in Basel aktiv und spielte eine wichtige Rolle in der reformatorischen Bewegung. Die Erwähnung seiner Wohnstätte deutet auf die enge Verbindung zwischen den Reformatoren und den humanistischen Kreisen in Basel hin.</note>
<note xml:i

In [29]:
calculate_openai_cost(tokens, (0.3,1.2))

0.006931800000000001

#### test 4
GPT can easily handle the long letter

In [54]:
# test letter
letter_id = 12260


footnote_df = pd.read_csv("../../data/footnote_df.csv")
# get the letter
with open(f"../../data/human/{letter_id}.xml") as infile:
  letter = infile.read()

In [57]:
instruct_prompt_no_fns(letter, 10, example_n=9)

[{'role': 'system',
  'content': 'You are a research assistant for a historian, specialized on the European reformation working on an edition of the correspondence of Heinrich Bullinger. Given a letter in TEI format, your task is to complete the missing footnote'},
 {'role': 'user',
  'content': 'Bitte schlage mir einen Text für Fussnote n=9 in folgendem Dokument vor:\n\n<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="file12260" type="Brief" source="HBBW-15" n="2213">\n\t<teiHeader xml:lang="de">\n\t\t<fileDesc>\n\t\t\t<titleStmt>\n\t\t\t\t<title subtype="file">Matthias Erb / Reichenweier an Heinrich Bullinger, 11. August [1545]</title>\n\t\t\t</titleStmt>\n\t\t\t<publicationStmt>\n\t\t\t\t<authority>Universität Zürich</authority>\n\t\t\t\t</publicationStmt>\n\t\t\t<sourceDesc>\n\t\t\t\t<bibl type="scan">Zürich StA</bibl>\n\t\t\t\t</sourceDesc>\n\t\t</fileDesc>\n\t\t</teiHeader>\n\t<text xml:lang="la">\n\t\t<body>\n\t\t\t<div xml:id="div1" corresp="regest1">\n\t\t\t\t<p>\n\t\t\t\t\t<s

It just repeats the suggested one, though. Maybe with another example?

In [55]:
tokens = []
for i in range(10):
    text, in_toks, out_toks = model.generate(instruct_prompt_no_fns(letter, 10, example_n=9))
    tokens.append((in_toks, out_toks))
    print(text)

<persName ref="p8098" cert="high">Graf Georg von Württemberg-Mömpelgard</persName>, der sich laut Z. 16 damals in <placeName ref="l569" cert="high">Worms</placeName> aufhielt. Seine Teilnahme am <placeName ref="l569" cert="high">Wormser</placeName> Reichstag ist in RTA JR XVIII-2 nicht belegt.
<persName ref="p8098" cert="high">Graf Georg von Württemberg-Mömpelgard</persName>, der sich laut Z. 16 damals in <placeName ref="l569" cert="high">Worms</placeName> aufhielt. Seine Teilnahme am <placeName ref="l569" cert="high">Wormser</placeName> Reichstag ist in RTA JR XVIII-2 nicht belegt.
<persName ref="p8098" cert="high">Graf Georg von Württemberg-Mömpelgard</persName>, der sich laut Z. 16 damals in <placeName ref="l569" cert="high">Worms</placeName> aufhielt. Seine Teilnahme am <placeName ref="l569" cert="high">Wormser</placeName> Reichstag ist in RTA JR XVIII-2 nicht belegt.
<persName ref="p8098" cert="high">Graf Georg von Württemberg-Mömpelgard</persName>, der sich laut Z. 16 damals in <

In [56]:
calculate_openai_cost(tokens, (3,6))

0.32058

In [58]:
tokens = []
for i in range(10):
    text, in_toks, out_toks = model.generate(instruct_prompt_no_fns(letter, 49, example_n=9))
    tokens.append((in_toks, out_toks))
    print(text)

<persName ref="p8127" cert="high">Pellicanum</persName> ist eine Abkürzung für den Theologen Konrad Pellikan (1478-1556), der als Reformator und Bibelübersetzer bekannt ist.
<persName ref="p8127" cert="high">Pellicanum</persName> ist eine Abkürzung für den Theologen Konrad Pellikan (1478-1556), der als Reformator und Bibelübersetzer bekannt ist.
<persName ref="p8127" cert="high">Pellicanum</persName> ist eine Anspielung auf den bekannten Theologen und Reformator Konrad Pellikan, der zu dieser Zeit in Zürich lebte und eng mit Heinrich Bullinger zusammenarbeitete.
<persName ref="p8127" cert="high">Pellicanum</persName> ist eine Anspielung auf den bekannten Theologen und Reformator Konrad Pellikan.
<persName ref="p8127" cert="high">Pellicanum</persName> ist eine Anspielung auf den Gelehrten und Theologen Konrad Pellikan, der zu dieser Zeit eine bedeutende Figur in der reformierten Bewegung war.
<persName ref="p8127" cert="high">Pellicanum</persName> ist eine Abkürzung für den Gelehrten un

In [59]:
calculate_openai_cost(tokens, (3,6))

0.317658

In [1]:
total = 0.317658+0.32058+0.006931800000000001+0.03447+0.0024186+0.0033066000000000003
total

0.6853650000000001

## Evaluate

| **Tests with Markup**             | BLEU  | ROUGE | roberta-base | xlm-roberta-base | wechsel-german | german-cased |
|------------------|-------|-------|--------------|------------------|----------------|--------------|
| Test 1          | **0.17** | **0.24**  | **0.85**      | **0.85**          | **0.82**       | **0.71**       |
| Test 2          | 0.00  | 0.01  | 0.70         | 0.69             | 0.66           | 0.50         |
| Test 3          | 0.16  | 0.15  | 0.84         | 0.83             | 0.79           | **0.71**         |

## installs and functions

In [ ]:
%pip install evaluate
%pip install transformers
%pip install bert-score
%pip install rouge_score

In [29]:
import re
from evaluate import load
import evaluate
from html import unescape
import bert_score
from tqdm import tqdm

bleu = load("bleu")
rouge = load("rouge")
bertscore = load("bertscore")

def remove_outer_note_tag(xml_str):
  """remove the outer note tag, since in the footnote_xml in the df it is still contained"""
  return re.match(r"<note [^>]*?>(.*?)</note>", xml_str, re.DOTALL).group(1)

def normalize_ws(text):
  """normalize whitespace"""
  return re.sub(r"\s+", " ", text).strip()

def remove_markup(sent):
  """Remove all markup from a sentence"""
  return unescape(re.sub(rf"<.*?>", "", sent))

###########
# Calculating BERT scores
###########
# Default of what layer to use for the representation
# The original module defines the default as
# "the number of layers tuned on WMT16 correlation data."
# I have to add some models, that are not in the original
# Maybe a todo: Test if the evaluation changes significantly based on this...
bert_score.utils.model2layers["google-bert/bert-base-german-cased"] = 9
bert_score.utils.model2layers["benjamin/roberta-base-wechsel-german"] = 9

def bertscore_f1_average(preds, refs, model_type):
  """calculate bertscore f1 average
  args:
    model_type: str, name of the model to use for the scoring"""

  results = bertscore.compute(predictions=preds, references=refs, model_type=model_type)
  f1 = results["f1"]
  return sum(f1) / len(f1)

def semantic_scores(predictions, references):
  """calculate semantic scores with BERT"""
  # calculate f1 average with different Bert models
  # the large models seem not to make much of a difference, but take more time
    # ->> but check it anyway sometime
  # The German models are more critical though
  # models = ["roberta-base", "xlm-roberta-base", "benjamin/roberta-base-wechsel-german",
    #         "google-bert/bert-base-german-cased"]
  models = ["benjamin/roberta-base-wechsel-german"]
  scores = {}
  # print(f"calculating Bert scores for {len(models)} models")
  for model in models:
    scores[model] = bertscore_f1_average(predictions, references, model)
  return scores

def all_metrics(predictions, references):

  all_results = {
      "with_markup": {},
      "without_markup": {}
  }
  # Calculate BLEU score
  bleu_results = bleu.compute(predictions=predictions, references=references)
  all_results["with_markup"]["bleu"] = bleu_results["bleu"]

  # Calculate ROUGE scores
  rouge_results = rouge.compute(predictions=predictions, references=references)
  all_results["with_markup"]["rouge"] = rouge_results["rouge1"]

  # Calculate Semantic score with BERT
  bertscore_results = semantic_scores(predictions=predictions, references=references)
  all_results["with_markup"]["bertscore"] = bertscore_results
  # print("\n".join([f"{model}: {score}" for model, score in bertscore_results.items()]))

  # removing markup
  predictions = [remove_markup(pred) for pred in predictions]
  references = [remove_markup(ref) for ref in references]

  # Calculate BLEU score
  bleu_results = bleu.compute(predictions=predictions, references=references)
  all_results["without_markup"]["bleu"] = bleu_results["bleu"]

  # Calculate ROUGE scores
  rouge_results = rouge.compute(predictions=predictions, references=references)
  all_results["without_markup"]["rouge"] = rouge_results["rouge1"]

  # Calculate Semantic score with BERT
  bertscore_results = semantic_scores(predictions=predictions, references=references)
  all_results["without_markup"]["bertscore"] = bertscore_results
  # print("\n".join([f"{model}: {score}" for model, score in bertscore_results.items()]))

  return all_results


In [6]:
!pip install requests_mock


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Test 1
instruction prompt to complete

In [2]:
def format_completion_model_output(x):
  """get a list of the generated fns without the end-tag"""
  generated_fns = x.split("\n")
  generated_fns = [re.sub(r"</note>.*$", "", fn).strip() for fn in generated_fns]  # remove end tag
  return generated_fns

In [ ]:
original_fn = """<persName ref="p1214" cert="high">Fries</persName>
und
<persName ref="p1283" cert="high">Geßner</persName>
waren in einem Brief
<persName ref="p8127" cert="high">Pellikans</persName>
vom 19. Februar 1533 an
<persName ref="p8418" cert="high">Myconius</persName>
empfohlen worden (Zürich StA, E II 358, 98).
</note>"""

x = """<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel near the residence of the printer <persName ref="p8418" cert="high">Myconius</persName>.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel near the residence of the printer <persName ref="p8418" cert="high">Myconius</persName>.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel near the printer <persName ref="p8418" cert="high">Myconius</persName>.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel where the letter was written.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel where the letter was written.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel where the letter was written.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel where the letter was written.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel near the residence of the printer <persName ref="p8418" cert="high">Myconius</persName>.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel near the residence of the printer <persName ref="p8418" cert="high">Myconius</persName>.</note>
<note xml:id="fn4" type="footnote" n="4"><placeName ref="l28" cert="high">Basileae</placeName> apud <persName ref="p8418" cert="high">Myconium</persName> refers to a location in Basel where the letter was written.</note>"""

generated_fns = format_completion_model_output(x)

# Normalize WS:
predictions = [normalize_ws(fn) for fn in generated_fns]
references = [normalize_ws(original_fn) for _ in range(len(generated_fns))]

In [ ]:
results_completion_test1 = all_metrics(predictions, references)

In [ ]:
import json
print(json.dumps(results_completion_test1, indent=4))

# Run Batches
**Note:** There was a problem with the createion of the QA prompt and I have to rerun it...
- [x] Instruct Add 
  - [x] example
  - [x] dev
  - [x] test
- [ ] Instruct QA 
  - [x] example
  - [ ] dev

In [4]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [5]:
import pandas as pd
import os
import jsonlines


# open the results file and save in a csv
def open_result_and_save_in_csv(outfile_name):
    response_file = outfile_name
    filepath, json_filename = os.path.split(outfile_name)
    csv_filename = f"{json_filename.split('.')[0]}.csv"
    csv_filepath = os.path.join(filepath, csv_filename)

    column_names = ["letter_id", "n_footnote", "generated_footnote", "prompt_tokens", "completion_tokens"]
    rows = []

    with jsonlines.open(response_file, "r") as infile:
        for line in infile:
            custom_id = line["custom_id"]
            letter_id, footnote_n = custom_id.split("_")
            generated_footnote = line["response"]["body"]["choices"][0]["message"]["content"]
            prompt_tokens = line["response"]["body"]["usage"]["prompt_tokens"]
            completion_tokens = line["response"]["body"]["usage"]["completion_tokens"]
            rows.append([letter_id, footnote_n, generated_footnote, prompt_tokens, completion_tokens])

    df = pd.DataFrame(columns=column_names)
    for row in rows:
        df.loc[len(df)] = row

    df.to_csv(csv_filepath, index=False)
    return df


def create_batch(model, prompt, split):
    prompt_file = f"{model}-{prompt}-{split}.jsonl"
    batch_input_file = client.files.create(
    file=open(f"../../data/prompts/{prompt}/{prompt_file}", "rb"),
    purpose="batch"
    )
    batch_input_file_id = batch_input_file.id

    description = prompt_file.split(".")[0]

    batch_object = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
        "description": description
        }
    )
    # print, to then put into the check and save function another day
    print(batch_object)
    print(prompt_file)


def check_batch_and_save(batch_id, prompt_file):
    batch_object = client.batches.retrieve(batch_id)  
    status = batch_object.status
    print(status)
    if status == "completed":
        output_file_id = batch_object.output_file_id
        file_response = client.files.content(output_file_id).content
        outfile_name = f"../../data/model_responses/gpt/{prompt_file}"
        with open(outfile_name, "wb") as outfile:
            outfile.write(file_response)
        return open_result_and_save_in_csv(outfile_name)

## Instruct Add 

### The test set

In [1]:
prompt_file = "gpt-4o-2024-08-06-instruct_add-test_human_eval.jsonl"

In [6]:
from openai import OpenAI
client = OpenAI(api_key=api_key)



batch_input_file = client.files.create(
  file=open(f"../../data/prompts/instruct_add/{prompt_file}", "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id

description = prompt_file.split(".")[0]

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": description
    }
)

Batch(id='batch_bLVivX0PHRvWdSY9opVRcW2q', completion_window='24h', created_at=1727190129, endpoint='/v1/chat/completions', input_file_id='file-UAb5MkWfxEi399BwnlbwX8UE', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1727276529, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18-instruct_add-test'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [5]:
batch_object = client.batches.retrieve("batch_bLVivX0PHRvWdSY9opVRcW2q")  # checking status
batch_object

Batch(id='batch_bLVivX0PHRvWdSY9opVRcW2q', completion_window='24h', created_at=1727190129, endpoint='/v1/chat/completions', input_file_id='file-UAb5MkWfxEi399BwnlbwX8UE', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1727191882, error_file_id=None, errors=None, expired_at=None, expires_at=1727276529, failed_at=None, finalizing_at=1727191603, in_progress_at=1727190133, metadata={'description': 'gpt-4o-mini-2024-07-18-instruct_add-test'}, output_file_id='file-pna9LXQ4KeQv5XAsM9nE5PEv', request_counts=BatchRequestCounts(completed=3239, failed=0, total=3239))

In [6]:

status = batch_object.status
print(status)
if status == "completed":
    output_file_id = batch_object.output_file_id
    file_response = client.files.content(output_file_id).content
    outfile_name = f"../../data/model_responses/gpt/{prompt_file}"
    with open(outfile_name, "wb") as outfile:
        outfile.write(file_response)
    df = open_result_and_save_in_csv(outfile_name)

completed


In [10]:
pd.set_option('display.max_colwidth', None)  # show whole text in the cells
df.head()

,letter_id,n_footnote,generated_footnote,prompt_tokens,completion_tokens
0,10038,10,"Die Frage, warum Christus nicht sofort nach seiner ersten Verheißung auf Erden erschienen ist, reflektiert eine theologische Debatte über das Wesen des göttlichen Plans und die Zeitlichkeit der Erlösung. In der christlichen Tradition wird häufig auf die Notwendigkeit verwiesen, dass das Kommen Christi in die Welt einen bestimmten göttlichen Zeitrahmen erfordert, der das Verständnis des Menschen überschreitet. Die erwähnten prophetischen Voraussagen bieten einen weiteren Kontext, um die Diskrepanz zwischen irdischem Leiden und göttlicher Erfüllung in der Heilsgeschichte zu interpretieren. Betrachtet man die Schriften des Neuen Testaments, insbesondere die Evangelien, wird deutlich, dass die zeitliche Offenbarung des Christus immer in Verbindung mit dem göttlichen Willen und dem Heilsplan steht.",7340,171
1,10038,11,"Der Hinweis auf Mose bezieht sich auf dessen Rolle als Prophet und Führer des Volkes Israel, das aus der ägyptischen Sklaverei befreit wurde. Die Referenz zu den Propheten kann auch die Erwartungen der Reformatoren widerspiegeln, die die Rückkehr zu einer ersten, unverdorbenen Form der Glaubensausübung forderten. Sie beziehen sich auf das biblische Verständnis von Offenbarung, das den Propheten (Mose und Paulus) als Vorbilder im Glauben sieht und deren Autorität zur Interpretation des göttlichen Willens betont.",7340,121
2,10038,12,"Die Frage nach dem Plan Gottes wird oft in der Theologie diskutiert. Bullinger deutet hier an, dass die Vorgaben der göttlichen Vorsehung und die Art und Weise, wie sie sich im Lauf der Geschichte entfalten, einen bestimmten Zweck haben, der oft schwer zu verstehen ist. So lässt sich auf Johannes Chrysostomus verweisen, der in seinen Schriften darauf hinweist, dass die Notwendigkeit des göttlichen Willens nicht immer für den Menschen erkennbar ist (vgl. ""In Matthaeum"", Homilie 19).",7340,116
3,10038,13,"In Anlehnung an die biblische Tradition spricht Bullinger hier die Idee an, dass die Propheten ""befreit"" wurden, um den Willen Gottes zu offenbaren und die Menschen auf den richtigen Weg zu führen. Der Verweis auf die ""deploratissimi"" Zustand der Menschen unter dem ""impiissimo Antichristi regno"" könnte auf die allgemeine Unzufriedenheit mit der religiösen Situation seiner Zeit hinweisen, insbesondere im Kontext der Reformation, bei der die Autorität der Kirche hinterfragt wurde. Bullinger drückt damit den Wunsch aus, die göttliche Wahrheit in einer Zeit der Verwirrung wiederherzustellen und die Menschen zu motivieren, Gott und seiner Botschaft treu zu bleiben.",7340,151
4,10038,14,"Die Betonung der göttlichen Inspiration und der prophetischen Freiheit, die im neuen Testament gefördert wird, zeigt sich besonders in <persName ref=""p1043"" cert=""high"">Johannes</persName> 1, 4-10 und <persName ref=""p18700"" cert=""high"">Paulus</persName> in seinem ersten Brief an die Thessalonicher 5, 20-21, wo das Prüfen aller Geister gefordert wird.<note xml:id=""fn14"" type=""footnote"" n=""14""></note>",7340,119


In [13]:
# calculate the cost of the batch:
tokens = [(row.prompt_tokens, row.completion_tokens) for _, row in df.iterrows()]
calculate_openai_cost(tokens, (0.075, 0.3))

2.544486149999998

In [7]:
# api_cost_logging(2.544486149999998, "gpt-4o-mini-2024-07-18-instruct_add-test")

### the dev set


In [4]:
prompt_file = "gpt-4o-mini-2024-07-18-instruct_add-dev.jsonl"
from openai import OpenAI
client = OpenAI(api_key=api_key)



batch_input_file = client.files.create(
  file=open(f"../../data/prompts/instruct_add/{prompt_file}", "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id

description = prompt_file.split(".")[0]

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": description
    }
)

Batch(id='batch_67026a6e980481909b404c63e2433966', completion_window='24h', created_at=1728211566, endpoint='/v1/chat/completions', input_file_id='file-Exypd6N5eKTHM3eG3gYJaZhh', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1728297966, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18-instruct_add-dev'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [13]:
batch_object = client.batches.retrieve("batch_67026a6e980481909b404c63e2433966")  # checking status with batch_id from above
status = batch_object.status
print(status)
if status == "completed":
    output_file_id = batch_object.output_file_id
    file_response = client.files.content(output_file_id).content
    outfile_name = f"../../data/model_responses/gpt/{prompt_file}"
    with open(outfile_name, "wb") as outfile:
        outfile.write(file_response)
    df = open_result_and_save_in_csv(outfile_name)

completed


In [14]:
pd.set_option('display.max_colwidth', None)  # show whole text in the cells
df.head()

,letter_id,n_footnote,generated_footnote,prompt_tokens,completion_tokens
0,10043,12,"In der Fußnote n=12 könnte folgender Text stehen:\n\n„Zuinglius (Huldrych Zwingli, 1484–1531) war ein wichtiger Reformator der Schweiz und zeitgenössischer Kollege von Bullinger. Seine theologischen Ansichten und seine Rolle bei der Reformation in Zürich hatten erheblichen Einfluss auf die Entwicklung der reformierten Tradition. Es ist bedeutsam, dass Bullinger ihn in diesem Brief erwähnt, was die enge Verbindung und den Austausch zwischen diesen führenden reformatorischen Persönlichkeiten unterstreicht.“",1732,112
1,10043,7,"Lactantius (Lucius Caecilius Firmianus Lactantius) war ein frühchristlicher Schriftsteller und Philosoph des 3. und 4. Jahrhunderts n. Chr., dessen Werke oft im Kontext der apologetischen Literatur zitiert werden. In seinem Werk „De vero cultu“ (Über die wahre Anbetung) kritisiert er die heidnischen Praktiken und betont die Wichtigkeit einer wahren, der christlichen Lehre entsprechenden Gottesdienstes. Insbesondere Kapitel 25 behandelt die Absurditäten und Ungerechtigkeiten, die durch falsche Anbetung und Götzendienst entstehen, was Bullinger in seinem Brief als relevante Argumentation gegen die Missstände seiner Zeit heranzieht.",1732,152
2,10043,8,"Die Erwähnung von ""Thrasonis"" bezieht sich auf den antiken Charakter Thrasonides aus den Komödien des römischen Dichters Plautus, der für seine Prahlerei bekannt ist. Bullinger verwendet hier den Namen symbolisch, um die Arroganz und Heuchelei der damaligen religiösen Führer zu kritisieren, die sich durch ihre Ansprüche an die Wahrheit und den Glauben auszeichnen, während sie in Wirklichkeit von Ungerechtigkeit und Sünde durchdrungen sind.",1732,106
3,10043,9,"Die Bezeichnung „harpigiis“ bezieht sich auf die damals verbreitete Auffassung, dass Papisten und ihre Praktiken als räuberisch oder schädlich angesehen wurden, ähnlich wie das Bild von Räubern oder Schurken. Es könnte hier auch auf die Vorstellung anspielen, dass solche Personen (Harpien) die Gläubigen ausrauben und ihnen das wahre Verständnis des Glaubens vorenthalten.",1732,87
4,10047,3,"Hier bezieht sich Leo Jud auf Luthers eigene Schwierigkeiten bei der Auslegung bestimmter biblischer Passagen, insbesondere im Hinblick auf die dogmatischen Differenzen zwischen der lutherischen und der reformierten Theologie. Luther hatte vor allem in seinen Schriften in den 1520er Jahren betont, dass es einige Stellen in den Schriften der Propheten oder den Aposteln gebe, die er nicht vollständig begreifen könne, was auf die Komplexität der theologischen Diskussionen in dieser Zeit hinweist. Diese Bemerkung ist sowohl eine Selbstkritik als auch ein Hinweis auf die Herausforderungen der theologischen Auseinandersetzung während der Reformation.",2734,136


In [15]:
# calculate the cost of the batch:
tokens = [(row.prompt_tokens, row.completion_tokens) for _, row in df.iterrows()]
calculate_openai_cost(tokens, (0.075, 0.3))

2.6265256499999974

In [8]:
# api_cost_logging(2.6265256499999974, "gpt-4o-mini-2024-07-18-instruct_add-dev")

## Instruct QA

### Example

In [17]:
create_batch("gpt-4o-mini-2024-07-18", "instruct_qa", "test")

Batch(id='batch_675adbffea5c8190b1c81876516fd818', completion_window='24h', created_at=1734007807, endpoint='/v1/chat/completions', input_file_id='file-CEPwBJLEXWuGF8aACPTyVS', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1734094207, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18-instruct_qa-test'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
gpt-4o-mini-2024-07-18-instruct_qa-test.jsonl


In [19]:
df = check_batch_and_save('batch_675adbffea5c8190b1c81876516fd818', "gpt-4o-mini-2024-07-18-instruct_qa-test.jsonl")

completed


In [21]:
df

,letter_id,n_footnote,generated_footnote,prompt_tokens,completion_tokens
0,11298,11,"Johannes Buchser, ein reformierter Theologe un...",1645,118
1,11298,12,Wilhelm von Fürstenberg (*circa 1480–1559) war...,1751,127
2,11298,3,"Siehe hierzu auch Johannes Calvin, *Institutio...",340,115
3,11298,4,"In diesem Kontext bezieht sich die ""ecclesiola...",502,88
4,11298,5,"Der Gedanke an ""Hoffnung gegen Hoffnung"" könnt...",692,99
...,...,...,...,...,...
107,12870,88,"<note xml:id=""fn88"" type=""footnote"" n=""88"">Der...",7582,128
108,12870,89,"<note xml:id=""fn89"" type=""footnote"" n=""89"">Der...",7743,110
109,12870,91,Franz I. von Frankreich (1494–1547) war währen...,7906,162
110,12870,97,"Die Quelle, die zusätzliche Informationen zu d...",8044,136


In [22]:
# calculate the cost of the batch:
tokens = [(row.prompt_tokens, row.completion_tokens) for _, row in df.iterrows()]
calculate_openai_cost(tokens, (0.075, 0.3))

0.04379715000000001

In [23]:
# api_cost_logging(0.04379715000000001,"gpt-4o-mini-2024-07-18-instruct_qa-example", comment="corrected prompt")

## dev set


In [44]:
create_batch("gpt-4o-mini-2024-07-18", "instruct_qa", "dev")

Batch(id='batch_6702bc5aecd88190aded2284a9fbdecc', completion_window='24h', created_at=1728232539, endpoint='/v1/chat/completions', input_file_id='file-lYa5o5ROtz0tzzFxDhMAy5Pj', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1728318939, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18-instruct_qa-dev'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
gpt-4o-mini-2024-07-18-instruct_qa-dev.jsonl


In [47]:
df = check_batch_and_save('batch_6702bc5aecd88190aded2284a9fbdecc', "gpt-4o-mini-2024-07-18-instruct_qa-dev.json")

completed


In [48]:
df

,letter_id,n_footnote,generated_footnote,prompt_tokens,completion_tokens
0,10043,12,"In Anmerkung 6 auf Seite 67 finden sich Informationen über die theologischen Positionen und Reformideen von Ulrich Zwingli, insbesondere seine Ansichten zur Eucharistie, seiner Rolle in der Zürcher Reformation sowie eine kurze biografische Skizze, die sowohl seine frühen Einflüsse als auch seine Konflikte mit Vertretern der lutherischen Lehre beleuchtet. Außerdem wird auf die Bedeutung seines Beitrags zur Reformbewegung im Kontext der religiösen und politischen Entwicklungen in der Schweiz eingegangen.",2106,108
1,10043,7,"Die zitierten Werke von Lactantius beziehen sich auf seine Schrift „Divinae Institutiones“ (Göttliche Institutionen), insbesondere auf Buch VI, Kapitel 25. In diesem Abschnitt behandelt Lactantius den „wahren Gottesdienst“ und kritisiert die Abgötterei sowie die Sünden, die im Namen falscher Kulte begangen werden. Diese Quelle liefert wertvolle Einsichten in die frühen christlichen Ansichten über Religion und Ethik und wird oft in theologischen Diskussionen zitiert.",1027,106
2,10043,8,"Im Kontext des Textes steht der Name ""Thraso"" symbolisch für eine prahlerische und spöttische Haltung. Thraso ist eine Figur aus dem Werk des römischen Dramatikers Terenz, die oft als Inbegriff des Wichtigtuers und Angeber dargestellt wird. Hier wird er als Beispiel für die Vortäuschung von Tugend und Religiosität genannt, während in Wahrheit eine Impietät und Heuchlerei vorherrscht, die die Unschuldigen verfolgt und die Kirche erdrosselt.",1479,116
3,10043,9,"Das Wort ""harpigium"" bezeichnet ein Werkzeug oder ein Gerät zum Greifen bzw. Einfangen, häufig im Kontext von Raub oder Diebstahl. Im Kontext des Satzes deutet es auf die mit Gewalt oder List verbundenen Mittel hin, die von den ""Papisten"" verwendet werden, um unrechtmäßig Besitz von den Glaubenden oder der Kirche zu ergreifen. Es impliziert somit, dass sie unehrliche Praktiken anwenden, um ihren Einfluss zu sichern und unschuldige Menschen zu verleumden.",1988,109
4,10047,3,"Luther verweist auf den Kommentar zum ersten Petrusbrief, den er in seiner Vorlesung behandelt hat, insbesondere in der Ausgabe der Weimarer Ausgaben (WA), wo er Schwierigkeiten in der Auslegung anmerkt.",461,46
...,...,...,...,...,...
3140,13134,7,"<persName ref=""p1419"" cert=""high"">Johannes Haab</persName> und <persName ref=""p6793"" cert=""high"">Itelhans Thumysen</persName> wurden auf die Tagsatzung am 22. November abgeordnet. Beide spielten eine entscheidende Rolle in der Kommunikation und Entscheidungsfindung während der politischen Verhandlungen in Baden, die im Zusammenhang mit der Reformation und den Konflikten innerhalb der Schweizer Eidgenossenschaft standen.",1447,102
3141,13134,8,"In Schreiben Nr. 3084 wird auf die politischen Spannungen zwischen den Reformierten und den katholischen Städten hingewiesen, die zu einer möglichen militärischen Auseinandersetzung führen könnten. Insbesondere werden die jüngsten Entscheidungen der Tagsatzung sowie die Reaktionen der Gesandten aus Zürich und anderen Städten thematisiert. Diese Informationen sind entscheidend, um die Dringlichkeit und den Inhalt der Warnung zu verstehen, da sie die Wahrnehmung von Bedrohungen und die Notwendigkeit zu diplomatischen Maßnahmen widerspiegeln.",1555,111
3142,13134,9,"<persName ref=""p18176"" cert=""high"">Kaspar Seidensticker</persName> war ein bedeutender reformierter Theologe und ein wichtiger Akteur in der reformierten Bewegung in Zürich während des 16. Jahrhunderts. In diesem Kontext spielt er eine relevante Rolle, da er oft in den Schriften und Correspondenzen, die die theologischen und politischen Auseinandersetzungen der Reformation betreffen, erwähnt wird. Sein Name steht exemplarisch für die engen Verbindungen zwischen den reformierten Gemeinden und deren Theologen, auf die Bullinger in seinen Schriften Bezug nimmt.",1702,122
3143,13135,7,"Die Form ""derort"" ist eine sprachliche Besonderheit des 16. Jahrhunderts

In [49]:
# calculate the cost of the batch:
tokens = [(row.prompt_tokens, row.completion_tokens) for _, row in df.iterrows()]
calculate_openai_cost(tokens, (0.075, 0.3))

1.0120045500000001

In [10]:
api_cost_logging(1.0120045500000001, "gpt-4o-mini-2024-07-18-instruct_qa-dev")

# get questions

In [10]:

create_batch("gpt-4o-mini-2024-07-18", "get_questions", "test")

Batch(id='batch_675ac069b49081908186e5792e4303c2', completion_window='24h', created_at=1734000745, endpoint='/v1/chat/completions', input_file_id='file-EHqBaE3Uu2tBeqwMsadW7g', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1734087145, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18-get_questions-test'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
gpt-4o-mini-2024-07-18-get_questions-test.jsonl


In [14]:
prompt_file = "gpt-4o-mini-2024-07-18-get_questions-test.jsonl"
df = check_batch_and_save("batch_675ac069b49081908186e5792e4303c2", prompt_file)

completed


In [15]:
pd.set_option('display.max_colwidth', None)  # show whole text in the cells
df

,letter_id,n_footnote,generated_footnote,prompt_tokens,completion_tokens
0,10038,10,Wie hat Augustinus in seinen Schriften seine Fehler während der theologischen Diskussionen begründet?,414,19
1,10038,11,Welche biblischen Stellen belegen die Aussagen von Moses und Paulus zur Prophetie?,234,17
2,10038,12,Welche biblischen Stellen unterstützen die Aussage über die prophetischen Gaben in 1 Korinther?,234,21
3,10038,13,"Auf welche biblische Quelle bezieht sich der Satz, um die Aussagen von Johannes zu stützen?",417,21
4,10038,14,Welche Bibelstellen beziehen sich auf den Umgang mit dem Heiligen Geist?,417,15
...,...,...,...,...,...
3493,13141,41,Warum wird die Information über Madenburg und die angebliche Hilfe für die Magdeburger in diesem Kontext erwähnt?,270,23
3494,13141,49,Welche Briefe könnten in der oben erwähnten Zeile gemeint sein?,145,15
3495,13141,5,Welche Quellen belegen die unterschiedlichen Ansichten zur Schriftlichkeit in religiösen Angelegenheiten?,206,18
3496,13141,7,Welche Informationen gibt es über Hans Rutishauser und Dorothea Diethelm gen. Münchin im Zusammenhang mit dem Turgöw?,441,28


In [16]:
# calculate the cost of the batch:
tokens = [(row.prompt_tokens, row.completion_tokens) for _, row in df.iterrows()]
calculate_openai_cost(tokens, (0.075, 0.3))

0.08821725000000011

In [11]:
# api_cost_logging(0.0024838500000000006, "gpt-4o-mini-2024-07-18-get_questions-example")

In [37]:
create_batch("gpt-4o-mini-2024-07-18", "get_questions", "dev")

Batch(id='batch_6702a13846088190b5b0a97b6f6dffdb', completion_window='24h', created_at=1728225592, endpoint='/v1/chat/completions', input_file_id='file-yAZVbIr1dbC1fPTWQoXjdLD5', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1728311992, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18-get_questions-dev'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
gpt-4o-mini-2024-07-18-get_questions-dev.jsonl


In [6]:
df = check_batch_and_save("batch_6702a13846088190b5b0a97b6f6dffdb", "gpt-4o-mini-2024-07-18-get_questions-dev.jsonl")

completed


In [7]:
pd.set_option('display.max_colwidth', None)  # show whole text in the cells
df

,letter_id,n_footnote,generated_footnote,prompt_tokens,completion_tokens
0,10043,12,Welche Informationen sind in Anmerkung 6 auf Seite 67 zu Zuinglium zu finden?,144,21
1,10043,5,"Was bedeutet das Wort ""excandeo""?",536,9
2,10043,7,Welche Quelle bezieht sich auf die Werke von Lactantius?,536,13
3,10043,8,"Welche Bedeutung hat der Name ""Thraso"" im Kontext des Textes?",536,16
4,10043,9,"Welche Bedeutung hat das Wort ""harpigium"" im Kontext des Satzes?",536,16
...,...,...,...,...,...
3400,13134,8,"Welche Informationen sind im Schreiben Nr. 3084 enthalten, auf das sich die Warnung bezieht?",213,21
3401,13134,9,Wer ist Kaspar Seidensticker und welche Relevanz hat er für den Text?,307,19
3402,13135,5,Wie bedeutend war die Angelegenheit?,380,8
3403,13135,7,"Welche sprachliche Besonderheit stammt aus dem 16. Jahrhundert und steht im Zusammenhang mit dem Begriff ""derort""?",380,23


In [43]:
# calculate the cost of the batch:
tokens = [(row.prompt_tokens, row.completion_tokens) for _, row in df.iterrows()]
calculate_openai_cost(tokens, (0.075, 0.3))

0.08705392499999999

In [12]:
api_cost_logging(0.08705392499999999,"gpt-4o-mini-2024-07-18-get_questions-dev")

In [5]:
create_batch("gpt-4o-mini-2024-07-18", "get_questions", "train")

Batch(id='batch_67087efc2d0c81909bb3806e0dd64ef5', completion_window='24h', created_at=1728610044, endpoint='/v1/chat/completions', input_file_id='file-hAT5uqR5xEvvGEtcHci9pJni', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1728696444, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'gpt-4o-mini-2024-07-18-get_questions-train'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
gpt-4o-mini-2024-07-18-get_questions-train.jsonl


In [5]:
df = check_batch_and_save("batch_67087efc2d0c81909bb3806e0dd64ef5", "gpt-4o-mini-2024-07-18-get_questions-train.jsonl")

completed


In [ ]:
pd.set_option('display.max_colwidth', None)  # show whole text in the cells
df

In [6]:
# calculate the cost of the batch:
tokens = [(row.prompt_tokens, row.completion_tokens) for _, row in df.iterrows()]
calculate_openai_cost(tokens, (0.075, 0.3))

0.6776942250000028

In [13]:
api_cost_logging(0.6776942250000028,"gpt-4o-mini-2024-07-18-get_questions-train")

# Cost

In [14]:
import pandas as pd
df = pd.read_csv("api_costs.csv")
print(sum(df["cost"]))
df

7.679410499999997


,date,experiment,comment,cost
0,NaN,informal_tests,NaN,0.685365
1,NaN,gpt-4o-mini-2024-07-18-instruct_add-test,NaN,2.544486
2,NaN,gpt-4o-mini-2024-07-18-instruct_add-dev,NaN,2.626526
3,NaN,gpt-4o-mini-2024-07-18-instruct_qa-example,faulty prompt,0.043797
4,NaN,gpt-4o-mini-2024-07-18-instruct_qa-dev,faulty prompt,1.012005
5,NaN,gpt-4o-mini-2024-07-18-get_questions-example,NaN,0.002484
6,NaN,gpt-4o-mini-2024-07-18-get_questions-dev,NaN,0.087054
7,NaN,gpt-4o-mini-2024-07-18-get_questions-train,NaN,0.677694
